In [21]:
import pandas as pd

housing_data = pd.read_csv("NY-House-Dataset.csv")

print("New York Housing Market dataset:")
print(housing_data.head())

rat_sightings_data = pd.read_csv("Rat_Sightings.csv")

print("\nNYC Rat Sightings dataset:")
print(rat_sightings_data.head())


New York Housing Market dataset:
                 TYPE      PRICE  BEDS       BATH  PROPERTYSQFT  \
0      Condo for sale     315000     2   2.000000        1400.0   
1      Condo for sale  195000000     7  10.000000       17545.0   
2      House for sale     260000     4   2.000000        2015.0   
3      Condo for sale      69000     3   1.000000         445.0   
4  Townhouse for sale   55000000     7   2.373861       14175.0   

                                             ADDRESS                    STATE  \
0                               2 E 55th St Unit 803       New York, NY 10022   
1  Central Park Tower Penthouse-217 W 57th New Yo...       New York, NY 10019   
2                                   620 Sinclair Ave  Staten Island, NY 10312   
3                            2 E 55th St Unit 908W33      Manhattan, NY 10022   
4                                        5 E 64th St       New York, NY 10065   

                                        MAIN_ADDRESS  \
0             2 E 55t

**New York Housing Market dataset:**
- There is a wide range of housing types listed, including condos, houses, and townhouses.
- Prices vary significantly, with some properties listed for millions of dollars and others for much less.
- The number of bedrooms and bathrooms also varies, indicating differences in property sizes and types.
- Geographical coordinates (latitude and longitude) are provided for each property, allowing for spatial analysis.

**NYC Rat Sightings dataset:**
- Rat sightings occur across various types of locations, including residential buildings, commercial buildings, and mixed-use buildings.
- There are missing values in the "Incident Address" column, indicating that not all sightings are associated with specific addresses.
- Rat sightings are reported across different zip codes and neighborhoods in New York City.
- The dataset includes geographical coordinates (latitude and longitude) for each sighting, facilitating spatial analysis and mapping.

**Observations:**
- There may be a relationship between the type of location (e.g., residential, commercial) and the frequency of rat sightings.
- It's possible that certain neighborhoods or zip codes have a higher incidence of rat sightings compared to others.
- There might be spatial correlations between areas with high house prices and areas with a high frequency of rat sightings, but further analysis is needed to confirm this.


**What feature/s would you like to be able to predict?** The geographical coordinates (latitude and longitude) of both house listings and rat sightings are crucial for spatial analysis. Mapping the distribution of house prices and rat sightings can help identify spatial patterns and correlations between the two features.

Geographical coordinates provide precise location information for each data point, allowing us to visualize the spatial distribution of house prices and rat sightings on a map. By plotting these coordinates, we can observe clusters or hotspots of high and low values for each feature across different neighborhoods or areas of New York City.



In [25]:
import pandas as pd

housing_data = pd.read_csv("NY-House-Dataset.csv")

missing_values_housing = housing_data.isnull().sum()
print("Missing values in New York Housing Market dataset:\n", missing_values_housing)

unique_types = housing_data['TYPE'].unique()
print("Unique values in the 'TYPE' column:\n", unique_types)

housing_data['TYPE'] = housing_data['TYPE'].astype('category')

housing_data.to_csv("updated_housing_data.csv", index=False)


rat_sightings_data = pd.read_csv("Rat_Sightings.csv")

missing_values = rat_sightings_data.isnull().sum()
print("\nMissing values:")
print(missing_values)

rat_sightings_data.dropna(subset=['Latitude', 'Longitude', 'Location'], inplace=True)

rat_sightings_data.to_csv("cleaned_rat_sightings_data.csv", index=False)



Missing values in New York Housing Market dataset:
 TYPE                           0
PRICE                          0
BEDS                           0
BATH                           0
PROPERTYSQFT                   0
ADDRESS                        0
STATE                          0
MAIN_ADDRESS                   0
ADMINISTRATIVE_AREA_LEVEL_2    0
LOCALITY                       0
SUBLOCALITY                    0
STREET_NAME                    0
LONG_NAME                      0
FORMATTED_ADDRESS              0
LATITUDE                       0
LONGITUDE                      0
dtype: int64
Unique values in the 'TYPE' column:
 ['Condo for sale' 'House for sale' 'Townhouse for sale' 'Co-op for sale'
 'Multi-family home for sale' 'For sale' 'Contingent' 'Land for sale'
 'Foreclosure' 'Pending' 'Coming Soon' 'Mobile house for sale'
 'Condop for sale']

Missing values:
Unique Key                       0
Location Type                    6
Incident Zip                   336
Incident Address      

In [30]:
from sklearn.model_selection import train_test_split

# Load housing data
housing_data = pd.read_csv("updated_housing_data.csv")
X_housing = housing_data.drop(columns=['PRICE'])  # Features
y_housing = housing_data['PRICE']  # Target variable

# Split housing data
X_train_housing, X_test_housing, y_train_housing, y_test_housing = train_test_split(X_housing, y_housing, test_size=0.2, random_state=42)

# Load rat sightings data
rat_sightings_data = pd.read_csv("cleaned_rat_sightings_data.csv")
X_rat_sightings = rat_sightings_data.drop(columns=['Location Type'])  # Features
y_rat_sightings = rat_sightings_data['Location Type']  # Target variable

# Split rat sightings data
X_train_rat, X_test_rat, y_train_rat, y_test_rat = train_test_split(X_rat_sightings, y_rat_sightings, test_size=0.2, random_state=42)
